# 2024 U.S. Presidential Election: Chances of Winning

In [2]:
# Define swing states with their electoral votes (ev)
states = {'AZ': 11, 'GA': 16, 'MI': 15, 'NV': 6, 'NC': 16, 'PA': 19, 'WI': 10};

# Define win probabilities, margins and, standard deviations for each of the 3 scenarios per swing state
input = {
    'latest': { # October 24, 2024
        'AZ': {'1': {'prob_harris': 0.30, 'margin': -2.7, 'sd': 5.2},
               '2': {'prob_harris': 0.26, 'margin': -4.0, 'sd': 6.3},
               '3': {'prob_harris': 0.27, 'margin': -3.4, 'sd': 5.5}},

        'GA': {'1': {'prob_harris': 0.43, 'margin': -1.1, 'sd': 6.0},
               '2': {'prob_harris': 0.43, 'margin': -1.3, 'sd': 7.2},
               '3': {'prob_harris': 0.43, 'margin': -1.2, 'sd': 6.4}},

        'MI': {'1': {'prob_harris': 0.53, 'margin':  0.5, 'sd': 6.1},
               '2': {'prob_harris': 0.29, 'margin': -4.1, 'sd': 7.3},
               '3': {'prob_harris': 0.39, 'margin': -1.8, 'sd': 6.4}},

        'NV': {'1': {'prob_harris': 0.58, 'margin':  0.8, 'sd': 4.3},
               '2': {'prob_harris': 0.47, 'margin': -0.5, 'sd': 6.3},
               '3': {'prob_harris': 0.51, 'margin':  0.1, 'sd': 4.9}},

        'NC': {'1': {'prob_harris': 0.49, 'margin': -0.2, 'sd': 4.2},
               '2': {'prob_harris': 0.20, 'margin': -4.7, 'sd': 5.6},
               '3': {'prob_harris': 0.30, 'margin': -2.4, 'sd': 4.6}},

        'PA': {'1': {'prob_harris': 0.61, 'margin':  1.3, 'sd': 5.0},
               '2': {'prob_harris': 0.26, 'margin': -4.1, 'sd': 6.2},
               '3': {'prob_harris': 0.40, 'margin': -1.4, 'sd': 5.3}},

        'WI': {'1': {'prob_harris': 0.63, 'margin':  1.8, 'sd': 5.2},
               '2': {'prob_harris': 0.26, 'margin': -4.4, 'sd': 6.8},
               '3': {'prob_harris': 0.41, 'margin': -1.3, 'sd': 5.7}}
    }
}

## **Version 1**: `random.random()`

Based on probabilities from https://github.com/maxspeicher/2024-us-presidential-election, as of October 24, 2024

In [3]:
import random

def run_election(scenario):
    ev_total_harris = 0
    ev_total_trump  = 0
    for state, info in input['latest'].items():
        # Simulate the election outcome for each state
        if random.random() < info[scenario]['prob_harris']:
            ev_total_harris += states[state]
        else:
            ev_total_trump += states[state]

    return {'harris': ev_total_harris, 'trump': ev_total_trump}

def monte_carlo_simulation(scenario, num_simulations=1000000):
    ev_needed_harris = 270 - 226  # Electoral votes Harris needs from swing states to reach 270
    ev_needed_trump  = 270 - 219  # Electoral votes Trump needs from swing states to reach 270
    wins_harris = 0
    wins_trump  = 0

    for _ in range(num_simulations):
        result = run_election(scenario)
        wins_harris += (result['harris'] >= ev_needed_harris)
        wins_trump  += (result['trump']  >= ev_needed_trump)

    return {'harris': wins_harris / num_simulations, 'trump': wins_trump / num_simulations}

# Run one Monte Carlo simulation for each scenario and print the probabilities of winning/tie
scenarios = {
    '1': '① Weighted high-quality polls',
    '2': '② With full 2016/20 polling error',
    '3': '③ With half 2016/20 polling error'
}

for scenario in ['1','2','3']:
  win_probability = monte_carlo_simulation(scenario)
  print(f"{scenarios[scenario]}\n===============================")
  print(f"Harris's probability of winning: {win_probability['harris']:.2%}")
  print(f"Trump's probability of winning: {win_probability['trump']:.2%}")
  print(f"Probability of tie: {(1-win_probability['trump']-win_probability['harris']):.2%}\n")

① Weighted high-quality polls
Harris's probability of winning: 59.10%
Trump's probability of winning: 39.71%
Probability of tie: 1.19%

② With full 2016/20 polling error
Harris's probability of winning: 16.72%
Trump's probability of winning: 82.06%
Probability of tie: 1.23%

③ With half 2016/20 polling error
Harris's probability of winning: 31.67%
Trump's probability of winning: 67.01%
Probability of tie: 1.32%



---

## **Version 2**: `np.random.normal(...)`

Based on polling margins and standard deviations from https://github.com/maxspeicher/2024-us-presidential-election, as of October 24, 2024

In [4]:
import random
import numpy as np

def run_election(scenario):
    ev_total_harris = 0
    ev_total_trump  = 0
    for state, info in input['latest'].items():
        # Generate random result based on average margin and standard deviation
        result = np.random.normal(info[scenario]['margin'], info[scenario]['sd'])

        # Determine winner of the state
        if result > 0:
            ev_total_harris += states[state]
        elif result < 0:
            ev_total_trump += states[state]

    return {'harris': ev_total_harris, 'trump': ev_total_trump}

def monte_carlo_simulation(scenario, num_simulations=1000000):
    ev_needed_harris = 270 - 226  # Electoral votes Harris needs from swing states to reach 270
    ev_needed_trump  = 270 - 219  # Electoral votes Trump needs from swing states to reach 270
    wins_harris = 0
    wins_trump  = 0

    for _ in range(num_simulations):
        result = run_election(scenario)
        wins_harris += (result['harris'] >= ev_needed_harris)
        wins_trump  += (result['trump']  >= ev_needed_trump)

    return {'harris': wins_harris / num_simulations, 'trump': wins_trump / num_simulations}

# Run one Monte Carlo simulation for each scenario and print the probabilities of winning/tie
scenarios = {
    '1': '① Weighted high-quality polls',
    '2': '② With full 2016/20 polling error',
    '3': '③ With half 2016/20 polling error'
}

for scenario in ['1','2','3']:
  win_probability = monte_carlo_simulation(scenario)
  print(f"{scenarios[scenario]}\n===============================")
  print(f"Harris's probability of winning: {win_probability['harris']:.2%}")
  print(f"Trump's probability of winning: {win_probability['trump']:.2%}")
  print(f"Probability of tie: {(1-win_probability['trump']-win_probability['harris']):.2%}\n")

① Weighted high-quality polls
Harris's probability of winning: 58.42%
Trump's probability of winning: 40.40%
Probability of tie: 1.18%

② With full 2016/20 polling error
Harris's probability of winning: 16.49%
Trump's probability of winning: 82.28%
Probability of tie: 1.23%

③ With half 2016/20 polling error
Harris's probability of winning: 31.23%
Trump's probability of winning: 67.42%
Probability of tie: 1.35%

